In [145]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ccxt
from pathlib import Path


In [146]:
def in_return_cov(df): 
    price = df.copy()
    price = (price / price.shift(1)) -1
    price.iloc[0,:] = 0 # Primeira linha = 0     
    
    returns = np.matrix(price)
    mean_returns = np.mean(returns, axis = 0)
    
    annual_returns = np.array([])
    for i in range(len(np.transpose(mean_returns))):
        annual_returns = np.append(annual_returns,(mean_returns[0,i]+1)**252-1)   
    
    cov = price.cov()*252
    
    return (annual_returns, np.matrix(cov))

#Carrega tabela de preços
def load_prices(symbol,
                index,
                start_date,
                end_date):
    
    dates = pd.date_range(start_date, end_date)
    dfBase = pd.DataFrame(index = dates)
    path = "C:/Users/Guilherme Overney/Desktop/DATA/{}.csv"
    
    dfIndex = pd.read_csv(path.format(index),index_col = "Date",
                          parse_dates = True,usecols = ['Date', 'Adj Close'], na_values=['nan'])
    
    dfIndex = dfIndex.dropna()
    dfIndex = dfIndex.rename(columns={'Adj Close': '{}'.format(index)})
    dfBase = dfBase.join(dfIndex, how = 'inner')
    
    
    for symbol in symbols:
        df_temp = pd.read_csv(path.format(symbol), index_col = "Date", parse_dates = True,
                              usecols = ['Date', 'Adj Close'], na_values=['nan'])
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        dfBase = dfBase.join(df_temp)
    
    dfBase.fillna(method = "ffill", inplace = True)
    dfBase.fillna(method = "bfill", inplace = True)
    
    return dfBase.drop('IBOV',axis = 1) #Tiro o índice da tabela de preços...

#Calcula o retorno de um portfólio de ativos
def port_return(W,r):
    return sum(W*r)

#Calcula o peso inicial do portfolio dado o tamanho de mercado do papel
def mkt_weights(weights):
    return np.array(weights) / sum(weights)

def risk_return(W,S,r,rf=0.06):
    var = np.dot(np.dot(W,S),np.transpose(W))
    port_r = port_return(W,r)
    return np.ndarray.item((port_r - rf) / var)

#Calcula a matriz de retorno implicito
def vector_equilibrium_return(S, W, r ):
    
    A = risk_return(W,S,r)
    
    return (np.dot(A,np.dot(S,W)))

def diago_omega(t, P, S):
    
    omega = np.dot(t,np.dot(P,np.dot(S,np.transpose(P))))
    
    for i in range(len(omega)):
        for y in range(len(omega)):
    return omega
    
#Black Litterman Model
def posterior_estimate_return(t,S,P,Q,PI):
    
    omega = diago_omega(t, P, S)
    
    parte_1 = t*np.dot(S,np.transpose(P))
    parte_2 = np.linalg.inv(np.dot(P*t,np.dot(S,np.transpose(P))) + omega)
    parte_3 = Q - np.dot(P,np.transpose(PI))
    
    return np.transpose(PI) + np.dot(parte_1,np.dot(parte_2,parte_3))

def posterior_covariance(t,S,P,PI):
    
    omega = diago_omega(t, P, S)
    
    parte_1 = t*np.dot(S,np.transpose(P))
    parte_2 = np.linalg.inv(t*np.dot(P,np.dot(S,np.transpose(P)))+omega)
    parte_3 = t*np.dot(P,S)

    return t*S - np.dot(parte_1,np.dot(parte_2,parte_3))

In [147]:
data=Path('Classification.csv')
cdf=pd.read_csv(data)
cdf.head()

,Unnamed: 0,symbol,Sharpe_Ratios,Circulating_Supply,Market_Cap,class
0,0,BCH/USD,-1.452502e-17,-1.452502e-17,1.016752e-16,0
1,1,BSV/USD,3.981345e-01,1.376221e-03,-4.705290e-01,0
2,2,BTC/USD,-6.388167e-01,1.329579e+00,1.995887e+00,0
3,3,DASH/USD,-2.202510e-01,-6.930314e-01,1.034104e+00,2
4,4,EOS/USD,7.540326e-02,1.697088e+00,3.793708e-02,0


In [148]:
symbols=[]
ave_risk=1
if ave_risk==1:
    symbols=cdf['symbol'].loc[(cdf['class']==2)|(cdf['class']==0)].to_list()
elif ave_risk==2:
    symbols=cdf['symbol'].to_list()
elif ave_risk==3:
    symbols=cdf['symbol'].loc[(cdf['class']==1)|(cdf['class']==0)].to_list()


In [149]:
def get_prices(symbols):
    i=0
    exchange = ccxt.kraken({
    'apiKey': KRAKEN_API_KEY,
    'secret': KRAKEN_SECRET_API_KEY,
    })
    since = exchange.parse8601('2018-01-01T00:00:00z')

    for symbol in symbols:
        if i==0:
            try:
                data = exchange.fetchOHLCV(symbol,'1d', since, params = {})
                header = ['Date', 'Open', 'High', 'Low', symbol, 'Volume']
                ticker_df = pd.DataFrame(data, columns=header)
                ticker_df['Date']=pd.to_datetime(ticker_df.Date/1000, unit='s')
                ticker_df.set_index('Date', inplace=True)
                ticker_df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
                i+=1
            except:
                continue
        else:
            try:
                data = exchange.fetchOHLCV(symbol,'1d', since, params = {})
                header = ['Date', 'Open', 'High', 'Low', symbol, 'Volume']
                df = pd.DataFrame(data, columns=header)
                df['Date']=pd.to_datetime(df.Date/1000, unit='s')
                df.set_index('Date', inplace=True)
                df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
                ticker_df=pd.concat([ticker_df, df], axis=1, join='inner')
            except:
                continue
    return(ticker_df)

In [150]:
ticker_df=get_prices(symbols)
ticker_df.head()

,BCH/USD,BTC/USD,DASH/USD,EOS/USD,ETC/USD,ETH/USD,LTC/USD,TRX/USD,XLM/USD,XMR/USD,XRP/USD,XTZ/USD
Date,,,,,,,,,,,,
2020-02-12,474.7,10354.6,133.395,5.3541,12.255,266.40,81.22,0.0,0.080356,95.13,0.30565,3.3556
2020-02-13,476.0,10241.6,131.250,5.3500,12.000,268.64,80.62,0.0,0.083658,92.22,0.32741,3.1345
2020-02-14,494.3,10370.8,133.064,5.3821,12.031,285.77,83.28,0.0,0.086660,94.74,0.33679,3.4785
2020-02-15,439.7,9904.6,120.422,4.7145,10.365,264.62,76.55,0.0,0.078707,89.08,0.30738,3.1847
2020-02-16,412.3,9923.9,111.076,4.3773,9.572,258.82,74.73,0.0,0.074408,87.85,0.29263,3.2213


In [151]:
tickers=ticker_df.columns.tolist()
data_df=pd.DataFrame()
data_df['tickers']=tickers
data_df['mrkcap']=0
data_df['returns']=0
data_df['SD']=0
data_df['ann_ret']=0
data_df['price']=0
data_df.set_index('tickers',inplace=True)
data_df.head()

,mrkcap,returns,SD,ann_ret,price
tickers,,,,,
BCH/USD,0,0,0,0,0
BTC/USD,0,0,0,0,0
DASH/USD,0,0,0,0,0
EOS/USD,0,0,0,0,0
ETC/USD,0,0,0,0,0


In [152]:
assets_return , S= in_return_cov(ticker_df)
data_df['returns']=assets_return

In [153]:
cdf.set_index('symbol', inplace=True)
cdf.head()

,Unnamed: 0,Sharpe_Ratios,Circulating_Supply,Market_Cap,class
symbol,,,,,
BCH/USD,0,-1.452502e-17,-1.452502e-17,1.016752e-16,0
BSV/USD,1,3.981345e-01,1.376221e-03,-4.705290e-01,0
BTC/USD,2,-6.388167e-01,1.329579e+00,1.995887e+00,0
DASH/USD,3,-2.202510e-01,-6.930314e-01,1.034104e+00,2
EOS/USD,4,7.540326e-02,1.697088e+00,3.793708e-02,0


In [154]:
pret=ticker_df.pct_change()
SD=pret.std()* np.sqrt(252)
ave_ret=pret.mean()* 252-.0062
last_price=ticker_df.iloc[-1,:]


In [155]:
for ticker in tickers:
    data_df.loc[ticker,'mrkcap']=cdf.loc[ticker, 'Market_Cap']
    data_df.loc[ticker,'ann_ret']=ave_ret.loc[ticker]
    data_df.loc[ticker,'price']=last_price.loc[ticker]
    data_df.loc[ticker,'SD']=SD.loc[ticker]
data_df.dropna(inplace=True)
clean_ticker=data_df.index.to_list()
data_df.head()

,mrkcap,returns,SD,ann_ret,price
tickers,,,,,
BCH/USD,1.016752e-16,-0.453293,0.991369,-0.613313,235.1000
BTC/USD,1.995887e+00,0.185549,0.805060,0.165191,9226.9000
DASH/USD,1.034104e+00,-0.401252,0.996207,-0.521987,71.6640
EOS/USD,3.793708e-02,-0.509516,0.928086,-0.722267,2.5964
ETC/USD,8.226090e-01,-0.415692,0.990228,-0.546509,6.3900


In [156]:
for ticker in tickers:
    if ticker not in clean_ticker:
        print('hi')
        ticker_df.drop([ticker], axis=1, inplace=True)
ticker_df.head()

hi


,BCH/USD,BTC/USD,DASH/USD,EOS/USD,ETC/USD,ETH/USD,LTC/USD,XLM/USD,XMR/USD,XRP/USD,XTZ/USD
Date,,,,,,,,,,,
2020-02-12,474.7,10354.6,133.395,5.3541,12.255,266.40,81.22,0.080356,95.13,0.30565,3.3556
2020-02-13,476.0,10241.6,131.250,5.3500,12.000,268.64,80.62,0.083658,92.22,0.32741,3.1345
2020-02-14,494.3,10370.8,133.064,5.3821,12.031,285.77,83.28,0.086660,94.74,0.33679,3.4785
2020-02-15,439.7,9904.6,120.422,4.7145,10.365,264.62,76.55,0.078707,89.08,0.30738,3.1847
2020-02-16,412.3,9923.9,111.076,4.3773,9.572,258.82,74.73,0.074408,87.85,0.29263,3.2213


In [157]:
data_df

,mrkcap,returns,SD,ann_ret,price
tickers,,,,,
BCH/USD,1.016752e-16,-0.453293,0.991369,-0.613313,235.100000
BTC/USD,1.995887e+00,0.185549,0.805060,0.165191,9226.900000
DASH/USD,1.034104e+00,-0.401252,0.996207,-0.521987,71.664000
EOS/USD,3.793708e-02,-0.509516,0.928086,-0.722267,2.596400
ETC/USD,8.226090e-01,-0.415692,0.990228,-0.546509,6.390000
ETH/USD,1.200624e+00,0.453340,1.002705,0.370419,238.530000
LTC/USD,-3.702333e-01,-0.432196,0.899897,-0.575288,44.320000
XLM/USD,-9.513388e-01,0.977296,0.925544,0.680974,0.092621
XMR/USD,-7.793335e-01,-0.095541,0.889628,-0.107263,68.630000


In [158]:
assets_return , S= in_return_cov(ticker_df)
assets_return

array([-0.45329282,  0.18554862, -0.40125244, -0.50951566, -0.41569201,
        0.45333973, -0.43219603,  0.97729605, -0.09554058, -0.31188433,
        0.23916945])

In [159]:
assets_return.shape

(11,)

In [160]:
S.shape

(11, 11)

In [161]:
W = mkt_weights(data_df['mrkcap'])
W.shape

(11,)

In [162]:
A = risk_return(W,S,data_df['returns'])
display(A)

-0.7774283113881989

In [163]:
PI = vector_equilibrium_return(S, W, data_df['returns'])
display(pd.DataFrame(np.transpose(PI), index=clean_ticker))

,0
BCH/USD,-0.560956
BTC/USD,-0.459436
DASH/USD,-0.532785
EOS/USD,-0.528117
ETC/USD,-0.544418
ETH/USD,-0.565057
LTC/USD,-0.501354
XLM/USD,-0.425647
XMR/USD,-0.462768
XRP/USD,-0.436747


In [166]:
N=len(clean_ticker)
Q = np.zeros((N,1))
P = np.zeros((N,N))
for n in range(len(clean_ticker)):
    P[n,n]=1
    Q[n,0]=data_df.iloc[n,3]

In [167]:
display(pd.DataFrame(Q, columns = ["views"]) )

display(pd.DataFrame(P,columns = clean_ticker) )

,views
0,-0.613313
1,0.165191
2,-0.521987
3,-0.722267
4,-0.546509
5,0.370419
6,-0.575288
7,0.680974
8,-0.107263
9,-0.382195


,BCH/USD,BTC/USD,DASH/USD,EOS/USD,ETC/USD,ETH/USD,LTC/USD,XLM/USD,XMR/USD,XRP/USD,XTZ/USD
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [168]:
t = 1 / len(ticker_df) # Medida de incerteza das estimativas de retornos pelo método Meucci
expc_return = posterior_estimate_return(t,S,P,Q,PI)
display(pd.DataFrame(expc_return, index = clean_ticker))

,0
BCH/USD,-0.306136
BTC/USD,-0.180208
DASH/USD,-0.328855
EOS/USD,-0.301989
ETC/USD,-0.343533
ETH/USD,-0.226036
LTC/USD,-0.253987
XLM/USD,-0.043540
XMR/USD,-0.188584
XRP/USD,-0.200677


In [169]:

post_cov = posterior_covariance(t,S,P,PI)
display(pd.DataFrame(post_cov, index = clean_ticker, columns = clean_ticker))

,BCH/USD,BTC/USD,DASH/USD,EOS/USD,ETC/USD,ETH/USD,LTC/USD,XLM/USD,XMR/USD,XRP/USD,XTZ/USD
BCH/USD,0.000894,0.000443,0.000354,0.000606,0.000583,0.000585,0.000534,0.000346,0.000447,0.000422,0.000429
BTC/USD,0.000443,0.000749,0.000295,0.000368,0.000246,0.000498,0.000385,0.000241,0.000463,0.000287,0.000417
DASH/USD,0.000354,0.000295,0.001896,0.000329,0.000298,0.000425,0.000315,0.000251,0.000342,0.000227,0.000459
EOS/USD,0.000606,0.000368,0.000329,0.000798,0.000602,0.000524,0.000493,0.000383,0.000429,0.000414,0.000298
ETC/USD,0.000583,0.000246,0.000298,0.000602,0.001291,0.000432,0.000548,0.000434,0.000362,0.000369,0.000240
ETH/USD,0.000585,0.000498,0.000425,0.000524,0.000432,0.000909,0.000524,0.000384,0.000419,0.000406,0.000648
LTC/USD,0.000534,0.000385,0.000315,0.000493,0.000548,0.000524,0.000681,0.000368,0.000440,0.000411,0.000442
XLM/USD,0.000346,0.000241,0.000251,0.000383,0.000434,0.000384,0.000368,0.001217,0.000347,0.000506,0.000483
XMR/USD,0.000447,0.000463,0.000342,0.000429,0.000362,0.000419,0.000440,0.000347,0.000911,0.000327,0.000405
XRP/USD,0.000422,0.000287,0.000227,0.000414,0.000369,0.000406,0.000411,0.000506,0.000327,0.000645,0.000327


In [170]:

cov_post_estimate = post_cov + S

display(pd.DataFrame(cov_post_estimate, index = clean_ticker, columns = clean_ticker))

,BCH/USD,BTC/USD,DASH/USD,EOS/USD,ETC/USD,ETH/USD,LTC/USD,XLM/USD,XMR/USD,XRP/USD,XTZ/USD
BCH/USD,0.977208,0.728153,0.769241,0.876415,0.887169,0.936065,0.847321,0.780122,0.798504,0.718163,0.894753
BTC/USD,0.728153,0.644580,0.612084,0.671362,0.657604,0.746562,0.662089,0.608307,0.651538,0.557858,0.724629
DASH/USD,0.769241,0.612084,0.987759,0.717303,0.720568,0.794001,0.701616,0.659193,0.680064,0.589898,0.795686
EOS/USD,0.876415,0.671362,0.717303,0.856451,0.840270,0.869652,0.790673,0.740117,0.747366,0.674691,0.815219
ETC/USD,0.887169,0.657604,0.720568,0.840270,0.975357,0.866151,0.813153,0.762243,0.745822,0.678141,0.813420
ETH/USD,0.936065,0.746562,0.794001,0.869652,0.866151,0.999671,0.853843,0.795818,0.802515,0.722503,0.949337
LTC/USD,0.847321,0.662089,0.701616,0.790673,0.813153,0.853843,0.805141,0.723713,0.735348,0.661163,0.828001
XLM/USD,0.780122,0.608307,0.659193,0.740117,0.762243,0.795818,0.723713,0.852189,0.689497,0.656555,0.806568
XMR/USD,0.798504,0.651538,0.680064,0.747366,0.745822,0.802515,0.735348,0.689497,0.787109,0.619983,0.789738
XRP/USD,0.718163,0.557858,0.589898,0.674691,0.678141,0.722503,0.661163,0.656555,0.619983,0.615644,0.698974


In [191]:
new_weigth = np.dot(np.transpose(expc_return),np.linalg.inv(A*cov_post_estimate))
weights_df=pd.DataFrame(new_weigth, columns = clean_ticker).T
weights_df

,0
BCH/USD,0.402713
BTC/USD,-0.022927
DASH/USD,0.594552
EOS/USD,0.641083
ETC/USD,0.540299
ETH/USD,-0.364556
LTC/USD,0.388004
XLM/USD,-1.404938
XMR/USD,-0.390877
XRP/USD,0.939251


In [192]:
port_df=weights_df.loc[weights_df[0]>0]
port_df

,0
BCH/USD,0.402713
DASH/USD,0.594552
EOS/USD,0.641083
ETC/USD,0.540299
LTC/USD,0.388004
XRP/USD,0.939251


In [193]:
port_df=port_df/port_df[0].sum()
port_df

,0
BCH/USD,0.114867
DASH/USD,0.169586
EOS/USD,0.182858
ETC/USD,0.154111
LTC/USD,0.110672
XRP/USD,0.267906


In [194]:
port_df.sum()
port_df.columns=['weights']

In [195]:
port_df['price']=0
port_df['return']=0
port_df['SD']=0
port_df['shares']=0
port_df['value']=0
port_tickers=port_df.index.to_list()
port_df

,weights,price,return,SD,shares,value
BCH/USD,0.114867,0,0,0,0,0
DASH/USD,0.169586,0,0,0,0,0
EOS/USD,0.182858,0,0,0,0,0
ETC/USD,0.154111,0,0,0,0,0
LTC/USD,0.110672,0,0,0,0,0
XRP/USD,0.267906,0,0,0,0,0


In [196]:
for ticker in clean_ticker:
    if ticker not in port_tickers:
        ticker_df.drop([ticker], axis=1, inplace=True)
ticker_df.head()

KeyError: "['BTC/USD'] not found in axis"

In [197]:
amount=1000000

for ticker in port_tickers:
    port_df.loc[ticker,'price']=data_df.loc[ticker,'price']
    port_df.loc[ticker,'SD']=data_df.loc[ticker,'SD']
    port_df.loc[ticker,'return']=data_df.loc[ticker,'ann_ret']
    port_df.loc[ticker,'shares']=(amount*port_df.loc[ticker, 'weights']//port_df.loc[ticker, 'price'])
    port_df.loc[ticker,'value']=(port_df.loc[ticker, 'shares']*port_df.loc[ticker, 'price'])  
left=amount-port_df['value'].sum()
port_df

,weights,price,return,SD,shares,value
BCH/USD,0.114867,235.1000,-0.613313,0.991369,488.0,114728.8000
DASH/USD,0.169586,71.6640,-0.521987,0.996207,2366.0,169557.0240
EOS/USD,0.182858,2.5964,-0.722267,0.928086,70427.0,182856.6628
ETC/USD,0.154111,6.3900,-0.546509,0.990228,24117.0,154107.6300
LTC/USD,0.110672,44.3200,-0.575288,0.899897,2497.0,110667.0400
XRP/USD,0.267906,0.1995,-0.382195,0.786826,1342886.0,267905.7570


In [198]:
left

In [203]:
wts=port_df['weights'].to_list()
wts=np.asarray(wts)
wts

array([0.11486715, 0.16958598, 0.18285814, 0.15411134, 0.11067163,
       0.26790576])

In [205]:
port_ret=ticker_df.pct_change()
port_ret.dropna(inplace=True)
port_ret['return']=port_ret.mul(wts,axis=1).sum(axis=1)
port_ret.head()


,BCH/USD,DASH/USD,EOS/USD,ETC/USD,ETH/USD,XRP/USD,return
Date,,,,,,,
2020-02-13,0.002739,-0.016080,-0.000766,-0.020808,0.008408,0.071193,0.014244
2020-02-14,0.038445,0.013821,0.006000,0.002583,0.063766,0.028649,0.022988
2020-02-15,-0.110459,-0.095007,-0.124041,-0.138476,-0.074011,-0.087324,-0.104408
2020-02-16,-0.062315,-0.077610,-0.071524,-0.076507,-0.021918,-0.047986,-0.060471
2020-02-17,0.000728,0.033986,-0.006328,0.008880,0.036319,-0.017975,0.005262


In [206]:
# =pret.std()* np.sqrt(252)
ann_ret=port_ret['return'].mean()* 252-.0062
ann_ret

-0.4366641704248239

In [209]:
cov_matrix = port_ret.iloc[:,0:-1].cov()
cov_matrix

,BCH/USD,DASH/USD,EOS/USD,ETC/USD,ETH/USD,XRP/USD
BCH/USD,0.003900,0.003071,0.003499,0.003542,0.003737,0.002867
DASH/USD,0.003071,0.003938,0.002864,0.002877,0.003170,0.002356
EOS/USD,0.003499,0.002864,0.003418,0.003354,0.003472,0.002694
ETC/USD,0.003542,0.002877,0.003354,0.003891,0.003458,0.002707
ETH/USD,0.003737,0.003170,0.003472,0.003458,0.003990,0.002885
XRP/USD,0.002867,0.002356,0.002694,0.002707,0.002885,0.002457


In [210]:
ann_port_std=round(((np.sqrt(np.dot(wts.T,np.dot
                                        (cov_matrix, wts)))
                                       * np.sqrt(252))),3)
ann_port_std

0.873

In [212]:
sharpe=ann_ret/ann_port_std
sharpe

-0.5001880531784925

In [ ]:
def make_data_df(coin_df, tickers_df, asset_returns, symbols):
    tickers=tickers_df.columns.tolist()
    data_df=pd.DataFrame()
    data_df['tickers']=''
    data_df['returns']=0
    data_df['mrkcap']=0
    data_df['price']=0

    clean_tickers, data_df=fill_data_df(coin_df, data_df, tickers_df, asset_returns, tickers)
    tickers_df=prune_tickers_df(clean_tickers, symbols, tickers_df)
    return(clean_tickers, data_df, tickers_df)


def fill_data_df(coin_df,data_df, tickers_df, asset_returns, tickers):
    data_df['tickers']=tickers
    data_df.set_index('tickers',inplace=True)
    data_df['returns']=asset_returns
    print(data_df)
    last_price=tickers_df.iloc[-1,:]
    print(last_price)
    for ticker in tickers:
        data_df.loc[ticker,'mrkcap']=coin_df.loc[ticker,'Market_Cap']
        data_df.loc[ticker,'price']=last_price.loc[ticker]
    data_df.dropna(inplace=True)
    data_df.set_index('tickers', inplace=True)
    clean_tickers=data_df.index.to_list()
        
    return(clean_ticker, data_df)

def prune_tickers_df(tickers_old, tickers_new, tickers_df):
    for ticker in tickers_old:
        if ticker not in tickers_new:
            ticker_df.drop([ticker], axis=1, inplace=True)
    return(ticker_df)